In [1]:
import numpy as np
import pyfftw
import scipy.fft
import matplotlib
import matplotlib.pyplot as pl
import nd2reader as nd2
import time

In [2]:
#define the filename
files = "../data/A1_s3001.nd2"

In [3]:
def radial_profile(data, centre):
    x, y = np.indices((data.shape))
    r = np.sqrt((x-centre[0])**2 + (y-centre[1])**2)
    r = r.astype(np.int)
    tbin = np.bincount(r.ravel(),data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin/nr
    return radialprofile

In [4]:
#reading in files based on their filetype
if files[-4:] == ".nd2":
    #read an nd2 file
    images = nd2.ND2Reader(files)
    width = images.metadata['width'] #the width of the image in pixels
    height = images.metadata['height'] #the height of the image in pixels
    # duration = images.metadata['experiment']['loops'][0]['duration'] #length of the video in milliseconds
    numberOfFrames = len(images) #number of frames in image
    print(numberOfFrames)
    images.iter_axes = 't' #this may need checked and may need to be changed to 't', the reader thinks z is time here though for my test data
elif files[-4:] == ".lif":
    #read a lif file
    print("foo")
else:
    print("unrecognised file type")
    exit()

5000


In [5]:
taus = np.arange(1,3) #range(int(numberOfFrames/2)) #the unit of tau is currently frames, we will convert it to s at the end
print(taus)

[1 2]


In [6]:
'''
This is the main bit which does all the looping over frames and lag times
'''
iTime = time.time()
for i in taus: #i is the lag time, generally labelled as tau or delta t
    gTau = np.zeros([width, height]) #initialise gTau to hold g(vec(q), tau) which will be radially averaged then saved
    imageDiffFTSquared = np.zeros([width, height]) #initialise zeros to hold the squared of the fourier transformed differences
    for j in range(numberOfFrames-i): #j is the initial frame in the difference calculation, usually labelled as t
        imageDiff = images[j+i] - images[j] #calculate the difference in pixel intensities between images
        imageDiffFT = np.fft.fft2(imageDiff) #fourier transform the difference
        imageDiffFTSquared += np.abs(imageDiffFT)**2 #for averaging this, add the square of the fourier transform to itself

    imageDiffFTSquared = np.fft.fftshift(imageDiffFTSquared)
    gTau = imageDiffFTSquared/(numberOfFrames-i)
    gTauRadial = radial_profile(gTau, (width/2., height/2.))
    #np.savetxt("gData/testExperimentRadial/test_tau{}.txt".format(i),gTauRadial)
    print(gTauRadial[10])
    print(time.time()-iTime) #benchmarking different fft algorithms

C:\Users\Maurits\AppData\Local\Temp\ipykernel_14432\197703533.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  r = r.astype(np.int)


333910033202007.0
95.30523324012756
389235327097185.4
193.70307445526123


In [7]:
'''
This is the main bit which does all the looping over frames and lag times
'''
iTime = time.time()
pyfftw.interfaces.cache.enable()
imageDiff = pyfftw.empty_aligned((height,width), dtype='complex64')
fft_object = pyfftw.builders.fft2(imageDiff, threads = 8,overwrite_input=True, planner_effort = 'FFTW_ESTIMATE')
for i in taus: #i is the lag time, generally labelled as tau or delta t

    gTau = np.zeros([width, height]) #initialise gTau to hold g(vec(q), tau) which will be radially averaged then saved
    imageDiffFTSquared = np.zeros([width, height]) #initialise zeros to hold the squared of the fourier transformed differences
    for j in range(numberOfFrames-i): #j is the initial frame in the difference calculation, usually labelled as t
        imageDiff = images[j+i] - images[j] #calculate the difference in pixel intensities between images
        imageDiffFTSquared += np.abs(fft_object(imageDiff))**2 #for averaging this, add the square of the fourier transform to itself

    imageDiffFTSquared = np.fft.fftshift(imageDiffFTSquared)
    gTau = imageDiffFTSquared/(numberOfFrames-i)
    gTauRadial = radial_profile(gTau, (width/2., height/2.))
    #np.savetxt("gData/testExperimentRadial/test_tau{}.txt".format(i),gTauRadial)
    print(gTauRadial[10])
    print(time.time()-iTime) #benchmarking different fft algorithms

C:\Users\Maurits\AppData\Local\Temp\ipykernel_14432\197703533.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  r = r.astype(np.int)


333910001828891.1
50.75640797615051
389235289416347.5
99.31342220306396
